<a href="https://colab.research.google.com/github/yr2387/E4511-2021-Rong/blob/main/Chembl_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install AMPL

In [ ]:
# get the Anaconda file 
! wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
! chmod +x Anaconda3-2019.10-Linux-x86_64.sh
! bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local

! time conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc

# install mordred, bravado and molvs
! time conda install -c conda-forge -y mordred bravado molvs

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

--2021-04-02 02:38:29--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import json
import os
import requests

# Import AMPL libraries
# 
#import atomsci.ddm.utils.data_curation_functions as dcf
import atomsci.ddm.utils.curate_data as curate_data
#import atomsci.ddm.pipeline.diversity_plots as dp
#import atomsci.ddm.pipeline.chem_diversity as cd
# Additional python libraries
import pandas as pd
import numpy as np
import getpass,os

In [ ]:
! wget https://raw.githubusercontent.com/yr2387/E4511-2021-Rong/main/Data/SLC6A2_ChEMBL_SMILES.csv

--2021-04-02 04:50:40--  https://raw.githubusercontent.com/yr2387/E4511-2021-Rong/main/Data/SLC6A2_ChEMBL_SMILES.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4019934 (3.8M) [text/plain]
Saving to: ‘SLC6A2_ChEMBL_SMILES.csv.3’

SLC6A2_ChEMBL_SMILE 100%[===================>]   3.83M  --.-KB/s    in 0.1s    

2021-04-02 04:50:40 (26.6 MB/s) - ‘SLC6A2_ChEMBL_SMILES.csv.3’ saved [4019934/4019934]



In [ ]:
orig_df = pd.read_csv('SLC6A2_ChEMBL_SMILES.csv', header = 0)
orig_df=orig_df.drop(columns=orig_df.iloc[:,0:1])
orig_df

,Molecule_ChEMBL_ID,Molecule_Name,Molecule_Max_Phase,Molecular_Weight,#RO5_Violations,AlogP,Compound_Key,Smiles,Standard_Type,Standard_Relation,Standard_Value,Standard_Units,pChEMBL_Value,Data_Validity_Comment,Comment,Uo_Units,Ligand_Efficiency_BEI,Ligand_Efficiency_LE,Ligand_Efficiency_LLE,Ligand_Efficiency_SEI,Potential_Duplicate,Assay_ChEMBL_ID,Assay_Description,Assay_Type,BAO_Format_ID,BAO_Label,Assay_Organism,Assay_Tissue_ChEMBL_ID,Assay_Tissue_Name,Assay_Cell_Type,Assay_Subcellular_Fraction,Assay_Parameters,Assay_Variant_Accession,Assay_Variant_Mutation,Target_ChEMBL_ID,Target_Name,Target_Organism,Target_Type,Document_ChEMBL_ID,Source_ID,Source_Description,Document_Journal,Document_Year,Cell_ChEMBL_ID,Properties,Standard_Value_nM,base_rdkit_smiles
0,CHEMBL3310477,NaN,0.0,358.21,1.0,5.00,2ak,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,Inhibition,'=',100.0,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL3364219,Inhibition of norepinephrine reuptake at human...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3352322,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2014.0,CHEMBL3307715,NaN,100.0,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F
1,CHEMBL4294989,NaN,0.0,394.95,1.0,5.47,30,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,Inhibition,'=',63.7,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL4276209,Inhibition of [3H]norepinephrine uptake at hum...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL4270660,1,Scientific Literature,Bioorg Med Chem,2018.0,NaN,NaN,63.7,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1
2,CHEMBL3323102,NaN,0.0,262.36,0.0,3.75,BDBM170456,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,Ki,'=',6.7,nM,8.17,NaN,314424,UO_0000065,31.16,0.56,4.42,42.95,False,CHEMBL3705836,Binding Assay: In order to evaluate the relati...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293E,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3638841,37,BindingDB Database,NaN,2015.0,NaN,NaN,6.7,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1
3,CHEMBL3310035,NaN,0.0,278.26,0.0,1.62,4,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,Activity,NaN,NaN,NaN,NaN,NaN,Not Active,NaN,NaN,NaN,NaN,NaN,False,CHEMBL3365809,Displacement of [3H]nisoxetine from NET (unkno...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3352407,1,Scientific Literature,J. Nat. Prod.,2014.0,CHEMBL3307715,NaN,NaN,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O
4,CHEMBL598346,NaN,0.0,289.32,0.0,4.22,1,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL1645860,Agonist activity at human cloned NET expressed...,F,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL1641456,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2011.0,CHEMBL3307715,NaN,28.0,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,CHEMBL4094228,NaN,0.0,440.39,0.0,4.45,2l,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,Inhibition,'>',55.0,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL4046309,Inhibition of re-uptake of [3H]-NE at human NE...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL4043179,1,Scientific Literature,Bioorg Med Chem,2017.0,CHEMBL3307715,NaN,55.0,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1
7530,CHEMBL1800685,NaN,0.0,475.51,1.0,7.36,NaN,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,Ki,'=',3270.0,nM,5.49,NaN,NaN,UO_0000065,11.54,0.21,-1.87,11.12,False,CHEMBL4508897,"Transporter, Norepinephrine (NET) E

In [ ]:
orig_df.columns = orig_df.columns.str.lower()

In [ ]:
orig_df

,molecule_chembl_id,molecule_name,molecule_max_phase,molecular_weight,#ro5_violations,alogp,compound_key,smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,data_validity_comment,comment,uo_units,ligand_efficiency_bei,ligand_efficiency_le,ligand_efficiency_lle,ligand_efficiency_sei,potential_duplicate,assay_chembl_id,assay_description,assay_type,bao_format_id,bao_label,assay_organism,assay_tissue_chembl_id,assay_tissue_name,assay_cell_type,assay_subcellular_fraction,assay_parameters,assay_variant_accession,assay_variant_mutation,target_chembl_id,target_name,target_organism,target_type,document_chembl_id,source_id,source_description,document_journal,document_year,cell_chembl_id,properties,standard_value_nm,base_rdkit_smiles
0,CHEMBL3310477,NaN,0.0,358.21,1.0,5.00,2ak,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,Inhibition,'=',100.0,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL3364219,Inhibition of norepinephrine reuptake at human...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3352322,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2014.0,CHEMBL3307715,NaN,100.0,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F
1,CHEMBL4294989,NaN,0.0,394.95,1.0,5.47,30,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,Inhibition,'=',63.7,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL4276209,Inhibition of [3H]norepinephrine uptake at hum...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL4270660,1,Scientific Literature,Bioorg Med Chem,2018.0,NaN,NaN,63.7,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1
2,CHEMBL3323102,NaN,0.0,262.36,0.0,3.75,BDBM170456,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,Ki,'=',6.7,nM,8.17,NaN,314424,UO_0000065,31.16,0.56,4.42,42.95,False,CHEMBL3705836,Binding Assay: In order to evaluate the relati...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293E,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3638841,37,BindingDB Database,NaN,2015.0,NaN,NaN,6.7,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1
3,CHEMBL3310035,NaN,0.0,278.26,0.0,1.62,4,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,Activity,NaN,NaN,NaN,NaN,NaN,Not Active,NaN,NaN,NaN,NaN,NaN,False,CHEMBL3365809,Displacement of [3H]nisoxetine from NET (unkno...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL3352407,1,Scientific Literature,J. Nat. Prod.,2014.0,CHEMBL3307715,NaN,NaN,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O
4,CHEMBL598346,NaN,0.0,289.32,0.0,4.22,1,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL1645860,Agonist activity at human cloned NET expressed...,F,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL1641456,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2011.0,CHEMBL3307715,NaN,28.0,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7529,CHEMBL4094228,NaN,0.0,440.39,0.0,4.45,2l,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,Inhibition,'>',55.0,%,NaN,NaN,NaN,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL4046309,Inhibition of re-uptake of [3H]-NE at human NE...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,HEK293,NaN,NaN,NaN,NaN,CHEMBL222,Norepinephrine transporter,Homo sapiens,SINGLE PROTEIN,CHEMBL4043179,1,Scientific Literature,Bioorg Med Chem,2017.0,CHEMBL3307715,NaN,55.0,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1
7530,CHEMBL1800685,NaN,0.0,475.51,1.0,7.36,NaN,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,Ki,'=',3270.0,nM,5.49,NaN,NaN,UO_0000065,11.54,0.21,-1.87,11.12,False,CHEMBL4508897,"Transporter, Norepinephrine (NET) E

In [ ]:
orig_df.columns

Index(['molecule_chembl_id', 'molecule_name', 'molecule_max_phase',
       'molecular_weight', '#ro5_violations', 'alogp', 'compound_key',
       'smiles', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'pchembl_value', 'data_validity_comment', 'comment',
       'uo_units', 'ligand_efficiency_bei', 'ligand_efficiency_le',
       'ligand_efficiency_lle', 'ligand_efficiency_sei', 'potential_duplicate',
       'assay_chembl_id', 'assay_description', 'assay_type', 'bao_format_id',
       'bao_label', 'assay_organism', 'assay_tissue_chembl_id',
       'assay_tissue_name', 'assay_cell_type', 'assay_subcellular_fraction',
       'assay_parameters', 'assay_variant_accession', 'assay_variant_mutation',
       'target_chembl_id', 'target_name', 'target_organism', 'target_type',
       'document_chembl_id', 'source_id', 'source_description',
       'document_journal', 'document_year', 'cell_chembl_id', 'properties',
       'standard_value_nm', 'base_rdkit_smiles'

In [ ]:
df = orig_df[['molecule_chembl_id','smiles', 'standard_type', 'standard_relation', 'standard_value',
              'standard_units', 'pchembl_value','base_rdkit_smiles','molecular_weight']]
df

,molecule_chembl_id,smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,base_rdkit_smiles,molecular_weight
0,CHEMBL3310477,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,Inhibition,'=',100.0,%,NaN,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,358.21
1,CHEMBL4294989,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,Inhibition,'=',63.7,%,NaN,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,394.95
2,CHEMBL3323102,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,Ki,'=',6.7,nM,8.17,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,262.36
3,CHEMBL3310035,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,Activity,NaN,NaN,NaN,NaN,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,278.26
4,CHEMBL598346,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,289.32
...,...,...,...,...,...,...,...,...,...
7529,CHEMBL4094228,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,Inhibition,'>',55.0,%,NaN,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,440.39
7530,CHEMBL1800685,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,Ki,'=',3270.0,nM,5.49,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,475.51
7531,CHEMBL3609637,COc1cc(-c2cn(C3CCc4c(F)cccc4N(CC(F)(F)F)C3=O)n...,Ki,'=',808.0,nM,6.09,COc1cc(-c2cn(C3CCc4c(F)cccc4N(CC(F)(F)F)C3=O)n...,514.48
7532,CHEMBL4533250,Cl.Clc1ccccc1OCc1cc(OC[C@@H]2CCCN2)no1,Ki,'=',2109.0,nM,5.68,Clc1ccccc1OCc1cc(OC[C@@H]2CCCN2)no1,345.23


In [ ]:
df.drop(index=df.loc[(df.molecular_weight>2000)].index)

,molecule_chembl_id,smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,base_rdkit_smiles,molecular_weight
0,CHEMBL3310477,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,Inhibition,'=',100.0,%,NaN,Fc1ccc(OC(CC2CNC2)c2ccc(Cl)c(Cl)c2)cc1F,358.21
1,CHEMBL4294989,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,Inhibition,'=',63.7,%,NaN,Clc1cccc(N2CCN(CCCCOc3ccc4ccccc4c3)CC2)c1,394.95
2,CHEMBL3323102,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,Ki,'=',6.7,nM,8.17,CN1Cc2ccccc2C(c2ccc3cc[nH]c3c2)C1,262.36
3,CHEMBL3310035,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,Activity,NaN,NaN,NaN,NaN,COC(=O)CCc1cc2cc(C)oc(=O)c2c(O)c1O,278.26
4,CHEMBL598346,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,289.32
...,...,...,...,...,...,...,...,...,...
7529,CHEMBL4094228,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,Inhibition,'>',55.0,%,NaN,Brc1ccccc1-c1nnnn1CCCN1CCC(Cc2ccccc2)CC1,440.39
7530,CHEMBL1800685,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,Ki,'=',3270.0,nM,5.49,O=C(O)c1cc(-c2ccc(C3CCNCC3)cc2)c2ccc(-c3ccc(C(...,475.51
7531,CHEMBL3609637,COc1cc(-c2cn(C3CCc4c(F)cccc4N(CC(F)(F)F)C3=O)n...,Ki,'=',808.0,nM,6.09,COc1cc(-c2cn(C3CCc4c(F)cccc4N(CC(F)(F)F)C3=O)n...,514.48
7532,CHEMBL4533250,Cl.Clc1ccccc1OCc1cc(OC[C@@H]2CCCN2)no1,Ki,'=',2109.0,nM,5.68,Clc1ccccc1OCc1cc(OC[C@@H]2CCCN2)no1,345.23


In [ ]:
print(sum(df['pchembl_value'].isna()) )
print(len(df) - sum(df['pchembl_value'].isna()))

len(orig_df)

3314
4220


7534

In [ ]:
dset_df = df[~(df.molecule_chembl_id.isna())  &
            (df.standard_units == 'nM') &
            ~df.standard_value.isna() &
             (df.standard_relation == "'='" ) &
             ~df.standard_relation.isna()&
             ~df.pchembl_value.isna()]

In [ ]:
dset_df = dset_df.loc[(df.standard_type == 'IC50') | (df.standard_type == 'EC50')]
dset_df

,molecule_chembl_id,smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,base_rdkit_smiles,molecular_weight
4,CHEMBL598346,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,289.32
7,CHEMBL465123,NCC[C@H](Oc1ccc(C(F)(F)F)cc1)c1ccccc1,IC50,'=',2806.0,nM,5.55,NCC[C@H](Oc1ccc(C(F)(F)F)cc1)c1ccccc1,295.30
9,CHEMBL498495,CNC[C@H]1COc2ccccc2[C@@H]1Oc1ccccc1C,IC50,'=',200.0,nM,6.70,CNC[C@H]1COc2ccccc2[C@@H]1Oc1ccccc1C,283.37
10,CHEMBL455447,CNC[C@@H]1Cc2ccccc2[C@@H]1Oc1ccccc1C,IC50,'=',10.0,nM,8.00,CNC[C@@H]1Cc2ccccc2[C@@H]1Oc1ccccc1C,267.37
16,CHEMBL4105127,CCCOC(=O)[C@@]12C[C@@H]1[C@@H](n1cnc3c(NC)nc(C...,EC50,'=',980.0,nM,6.01,CCCOC(=O)[C@@]12C[C@@H]1[C@@H](n1cnc3c(NC)nc(C...,487.97
...,...,...,...,...,...,...,...,...,...
7510,CHEMBL4563913,N=C(N)NCCCCCC(=O)O[C@H]1C[C@H](n2cc([125I])c(=...,IC50,'=',28700.0,nM,4.54,N=C(N)NCCCCCC(=O)O[C@H]1C[C@H](n2cc(I)c(=O)[nH...,507.30
7513,CHEMBL4083724,c1ccc(CCCN2CCN(CCCCn3nnnc3C(c3ccccc3)c3ccccc3)...,IC50,'=',99.0,nM,7.00,c1ccc(CCCN2CCN(CCCCn3nnnc3C(c3ccccc3)c3ccccc3)...,494.69
7518,CHEMBL4455262,Fc1cc(Cl)ccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2cccc...,IC50,'=',1100.0,nM,5.96,Fc1cc(Cl)ccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2cccc...,491.01
7520,CHEMBL4522348,Clc1ccccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2ccccc2)CC1,IC50,'=',670.0,nM,6.17,Clc1ccccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2ccccc2)CC1,473.02


In [ ]:
sum(df.base_rdkit_smiles.isna())

0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dset_df.to_csv('drive/MyDrive/Columbia_E4511/CHEMBL_DROP.csv')

In [ ]:
dset_df=dset_df.rename(columns={'pchembl_value':'PXC50'})
dset_df

,molecule_chembl_id,smiles,standard_type,standard_relation,standard_value,standard_units,PXC50,base_rdkit_smiles,molecular_weight
4,CHEMBL598346,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,EC50,'=',28.0,nM,7.55,Fc1ccccc1Oc1cccc(F)c1C1CCNCC1,289.32
7,CHEMBL465123,NCC[C@H](Oc1ccc(C(F)(F)F)cc1)c1ccccc1,IC50,'=',2806.0,nM,5.55,NCC[C@H](Oc1ccc(C(F)(F)F)cc1)c1ccccc1,295.30
9,CHEMBL498495,CNC[C@H]1COc2ccccc2[C@@H]1Oc1ccccc1C,IC50,'=',200.0,nM,6.70,CNC[C@H]1COc2ccccc2[C@@H]1Oc1ccccc1C,283.37
10,CHEMBL455447,CNC[C@@H]1Cc2ccccc2[C@@H]1Oc1ccccc1C,IC50,'=',10.0,nM,8.00,CNC[C@@H]1Cc2ccccc2[C@@H]1Oc1ccccc1C,267.37
16,CHEMBL4105127,CCCOC(=O)[C@@]12C[C@@H]1[C@@H](n1cnc3c(NC)nc(C...,EC50,'=',980.0,nM,6.01,CCCOC(=O)[C@@]12C[C@@H]1[C@@H](n1cnc3c(NC)nc(C...,487.97
...,...,...,...,...,...,...,...,...,...
7510,CHEMBL4563913,N=C(N)NCCCCCC(=O)O[C@H]1C[C@H](n2cc([125I])c(=...,IC50,'=',28700.0,nM,4.54,N=C(N)NCCCCCC(=O)O[C@H]1C[C@H](n2cc(I)c(=O)[nH...,507.30
7513,CHEMBL4083724,c1ccc(CCCN2CCN(CCCCn3nnnc3C(c3ccccc3)c3ccccc3)...,IC50,'=',99.0,nM,7.00,c1ccc(CCCN2CCN(CCCCn3nnnc3C(c3ccccc3)c3ccccc3)...,494.69
7518,CHEMBL4455262,Fc1cc(Cl)ccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2cccc...,IC50,'=',1100.0,nM,5.96,Fc1cc(Cl)ccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2cccc...,491.01
7520,CHEMBL4522348,Clc1ccccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2ccccc2)CC1,IC50,'=',670.0,nM,6.17,Clc1ccccc1N1CCN(CCCn2nnnc2C(c2ccccc2)c2ccccc2)CC1,473.02


In [ ]:
import os

# From our dataframe, we are working with the PIC50 column 
data=dset_df

column = 'PXC50'

# tolerance: Percentage of individual respsonse values allowed to different from 
# the average to be included in averaging
tolerance = 10

# list_bad_duplicates: Print structures with bad duplicates
list_bad_duplicates = 'Yes'

# max_std: Maximum allowed standard deviation for computed average response value
# NOTE: In this example, we set this value very high to disable this feature
max_std = 1

# compound_id: Compound ID column
compound_id = 'molecule_chembl_id'

# smiles_col: SMILES column
smiles_col = 'base_rdkit_smiles'

# Here we are creating a new dataframe, called check_df
check_df = curate_data.average_and_remove_duplicates(column, tolerance, 
                                                       list_bad_duplicates, 
                                                       data, max_std, 
                                                       compound_id=compound_id, 
                                                       smiles_col=smiles_col)

Bad duplicates removed from dataset
Dataframe size (2286, 13)
List of 'bad' duplicates removed
     molecule_chembl_id  PXC50  VALUE_NUM_mean   Perc_Var  VALUE_NUM_std
2104      CHEMBL1172928   5.00           6.000  16.666667       1.414214
924       CHEMBL1172928   7.00           6.000  16.666667       1.414214
1250       CHEMBL180672   9.70           8.200  18.292683       2.121320
189       CHEMBL2112075   6.70           8.200  18.292683       2.121320
1510       CHEMBL219116   7.96           6.905  15.278783       1.491995
872        CHEMBL219116   5.85           6.905  15.278783       1.491995
528        CHEMBL219224   5.50           6.480  15.123457       1.385929
117        CHEMBL219224   7.46           6.480  15.123457       1.385929
1704       CHEMBL259209   7.11           7.180   0.974930       1.219147
217        CHEMBL259209   7.39           7.180   2.924791       1.219147
2061       CHEMBL259209   7.11           7.180   0.974930       1.219147
925        CHEMBL259209   9.1

In [ ]:
old_compound_id='base_rdkit_smiles'
new_compound_id='base_rdkit_smiles'

# Takes all the compounds that aren't part of the curated data frame and prints them
reject=data[~data[old_compound_id].isin(check_df[new_compound_id])]
reject

,molecule_chembl_id,smiles,standard_type,standard_relation,standard_value,standard_units,PXC50,base_rdkit_smiles,molecular_weight
49,CHEMBL486278,COc1ccc(CC(C)(C)NC[C@H](O)COc2cccc3[nH]c(=O)[n...,IC50,'=',600.0000,nM,6.22,COc1ccc(CC(C)(C)NC[C@H](O)COc2cccc3[nH]c(=O)[n...,385.46
339,CHEMBL219224,CC(c1ccc(Cl)cc1)C1CCCCN1,IC50,'=',35.0000,nM,7.46,CC(c1ccc(Cl)cc1)C1CCCCN1,223.75
591,CHEMBL2112075,CC(C)(Cc1ccc2ccccc2c1)NC[C@@H](O)COc1cccc(Cl)c...,IC50,'=',200.0000,nM,6.70,CC(C)(Cc1ccc2ccccc2c1)NC[C@@H](O)COc1cccc(Cl)c...,445.39
687,CHEMBL259209,CCN(CC)C(=O)C1(c2ccccc2)CC1CN,IC50,'=',41.0000,nM,7.39,CCN(CC)C(=O)C1(c2ccccc2)CC1CN,246.35
1072,CHEMBL3323095,CN1Cc2ccccc2C(c2ccc3sccc3c2)C1,IC50,'=',3.4000,nM,8.47,CN1Cc2ccccc2C(c2ccc3sccc3c2)C1,279.41
1516,CHEMBL486278,COc1ccc(CC(C)(C)NC[C@H](O)COc2cccc3[nH]c(=O)[n...,IC50,'=',0.6000,nM,9.22,COc1ccc(CC(C)(C)NC[C@H](O)COc2cccc3[nH]c(=O)[n...,385.46
1861,CHEMBL219224,CC(c1ccc(Cl)cc1)C1CCCCN1,IC50,'=',3200.0000,nM,5.50,CC(c1ccc(Cl)cc1)C1CCCCN1,223.75
1946,CHEMBL259209,CCN(CC)C(=O)C1(c2ccccc2)CC1CN,IC50,'=',5500.0000,nM,5.26,CCN(CC)C(=O)C1(c2ccccc2)CC1CN,246.35
1965,CHEMBL3323095,CN1Cc2ccccc2C(c2ccc3sccc3c2)C1,IC50,'=',350.0000,nM,6.46,CN1Cc2ccccc2C(c2ccc3sccc3c2)C1,279.41
2043,CHEMBL612,C[C@H](N)Cc1ccccc1,IC50,'=',630.9600,nM,6.20,C[C@H](N)Cc1ccccc1,135.21


In [ ]:
column='PXC50'; #'standard_value'
list_bad_duplicates='Yes'

# Actually compiles all the data with the specifies column and tells what columns to include 
# (inchi_key, smiles, and relation)
temp_df=curate_data.aggregate_assay_data(data, 
                                         value_col=column, 
                                         output_value_col=None,
                                         label_actives=True,
                                         active_thresh=6,
                                         id_col='molecule_chembl_id', 
                                         smiles_col='base_rdkit_smiles', 
                                         relation_col='standard_relation')

#Removes all duplicates from the data frame and puts it in the final, curated dataframe
curated_df = temp_df[~temp_df.isin([np.inf]).any(1)]

0 entries in input table are missing SMILES strings
1909 unique SMILES strings are reduced to 1907 unique base SMILES strings


In [ ]:
curated_df.to_csv('drive/MyDrive/Columbia_E4511/CHEMBL_curated.csv')